# Import socle NLP

In [1]:
import sys
import os
folder = ''  # Change this to folder containing socle_nlp repo
sys.path.insert(0, os.path.dirname(os.getcwd()))

In [ ]:
# Silencing TreeTagger FutureWarning
import warnings
with warnings.catch_warnings():
    warnings.simplefilter(action='ignore', category=FutureWarning)
    from socle_nlp.text_processing.lemmatizer import Lemmatizer

In [3]:
import pandas as pd

C:\HOMEWARE\Anaconda3-Windows-x86_64\envs\socle_nlp_test\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


# Préparation des données

https://github.com/etalab-ia/piaf-code/raw/master/piaf-v1.0.json

In [4]:
import json
path = folder + 'Datasets\\piaf-v1.0.json'
with open(path) as f:
    piaf_json = json.load(f)['data']
titles = [sample['title'] for sample in piaf_json]
categories = [sample['categorie'] for sample in piaf_json]
contexts = [sample['paragraphs'][0]['context'] for sample in piaf_json]
piaf = pd.DataFrame({'id' : [*range(len(piaf_json))],
                        'title': titles,
                        'categorie': categories,
                        'context': contexts}).set_index('id')
piaf

,title,categorie,context
id,,,
0,Sport,Sport,Les dépenses des ménages représentent plus de ...
1,Jakob Böhme,Religion,"Mais ce régime de travail, combiné aux inquiét..."
2,Yoweri Museveni,Société,"Dans le droit fil du consensus de Washington, ..."
3,Les Ulis,Géographie,Le ruisseau Le Rouillon prend sa source dans l...
4,<i>Terminator 2 : Le Jugement dernier</i>,Arts,"Parallèlement, arrive du futur un autre person..."
...,...,...,...
186,Marine indienne,Société,"La marine indienne (sanskrit : भारतीय नौसेना, ..."
187,Los Fabulosos Cadillacs,Arts,"Un premier album studio, intitulé Bares y fond..."
188,Traduction génétique,Sciences,Le processus de traduction peut être bloqué pa...


In [5]:
from sklearn.pipeline import Pipeline
from socle_nlp.text_processing.all_transformers import RemovePunctTransformer, RemovePatternTransformer

rm_punct = RemovePunctTransformer(in_col='context', out_col='context')

spacy_lemma_sm = Lemmatizer('context', 'spacy_sm', engine='spacy', engine_conf='fr_core_news_sm')


spacy_lemma_md = Lemmatizer('context', 'spacy_md', engine='spacy', engine_conf='fr_core_news_md')


spacy_lemma_lg = Lemmatizer('context', 'spacy_lg', engine='spacy', engine_conf='fr_core_news_sm')


lefff_lemma = Lemmatizer('context', 'lefff', engine='Lefff', engine_conf='fr_core_news_sm')

treetagger_lemma = Lemmatizer('context', 'treeTagger', engine='TreeTagger',
                                engine_conf={'TAGDIR': os.path.join(folder, "TreeTagger"),
                                           # Replace with variable where TreeTagger will be downloaded and extracted
                                           'TAGLANG': 'fr',
                                           'TAGPARFILE': 'french.par'
                                           })

rm_pattern = RemovePatternTransformer(in_col='context', out_col='context', rm_pattern_regex=['[^\x00-\xFF]'])

lemmas_pipe = Pipeline([('rm_pattern', rm_pattern),
                        ('spacy_sm', spacy_lemma_sm),
                        ('spacy_md', spacy_lemma_md),
                        ('spacy_lg', spacy_lemma_lg),
                        ('lefff', lefff_lemma),
                        ('treetagger', treetagger_lemma)
                       ])

In [6]:
lemmas_pipe.fit_transform(piaf)

,title,categorie,context,spacy_sm,spacy_md,spacy_lg,lefff,treeTagger
id,,,,,,,,
0,Sport,Sport,Les dépenses des ménages représentent plus de ...,le dépense de ménage représenter plus de 50 po...,le dépense de ménage représenter plus de 50 po...,le dépense de ménage représenter plus de 50 po...,le dépense des ménage représenter plus de 50 p...,le dépense du ménage représenter plus de @card...
1,Jakob Böhme,Religion,"Mais ce régime de travail, combiné aux inquiét...","mais ce régime de travail , combiner à inquiét...","mais ce régime de travail , combiner à inquiét...","mais ce régime de travail , combiner à inquiét...","mais ce régime de travail , combiné aux inquié...","mais ce régime de travail , combiner au inquié..."
2,Yoweri Museveni,Société,"Dans le droit fil du consensus de Washington, ...","dans le droit fil de consensus de Washington ,...","dans le droit fil de consensus de Washington ,...","dans le droit fil de consensus de Washington ,...","dans le droit fil du consensus de washington ,...","dans le droit fil du consensus de Washington ,..."
3,Les Ulis,Géographie,Le ruisseau Le Rouillon prend sa source dans l...,le ruisseau le Rouillon prendre son source dan...,le ruisseau le Rouillon prendre son source dan...,le ruisseau le Rouillon prendre son source dan...,le ruisseau le rouillon prendre son source dan...,le ruisseau le Rouillon prendre son source dan...
4,<i>Terminator 2 : Le Jugement dernier</i>,Arts,"Parallèlement, arrive du futur un autre person...","parallèlement , arrive de futur un autre perso...","parallèlement , arrive de futur un autre perso...","parallèlement , arrive de futur un autre perso...","parallèlement , arriver du futur un autre pers...","parallèlement , arriver du futur un autre pers..."
...,...,...,...,...,...,...,...,...
186,Marine indienne,Société,"La marine indienne (sanskrit : , Bharatiya Na...","le marine indien ( sanskrit : , Bharatiya na...","le marine indien ( sanskrit : , Bharatiya Na...","le marine indien ( sanskrit : , Bharatiya na...","le marine indien ( sanskrit : , bharatiya na...","le marine indien ( sanskrit : , Bharatiya Nau ..."
187,Los Fabulosos Cadillacs,Arts,"Un premier album studio, intitulé Bares y fond...","un premier album studio , intituler Bares y fo...","un premier album studio , intituler bare y fon...","un premier album studio , intituler Bares y fo...","un premier album studio , intitulé bares y fon...","un premier album studio , intitulé Bares y fon..."
188,Traduction génétique,Sciences,Le processus de traduction peut être bloqué pa...,le processus de traduction pouvoir être bloque...,le processus de traduction pouvoir être bloque...,le processus de traduction pouvoir être bloque...,le processus de traduction pouvoir être bloqué...,le processus de traduction pouvoir être bloque...


# Comparing lemmas for 2 engines

In [7]:
engine1 = 'spacy_sm'
engine2 = 'lefff'

In [8]:
comparison = pd.Series(index=piaf.index, dtype=object)
for i, row in piaf[[engine1, engine2]].iterrows():
    z = [*zip(row.spacy_sm.lower().split(), row.lefff.lower().split())]
    diff = []
    for duo in z:
        if duo[0] == duo[1]:
            pass
        else:
            diff.append('|'.join([*duo]))
    comparison.at[i] = diff  

In [9]:
piaf['comparison'] = comparison
piaf

,title,categorie,context,spacy_sm,spacy_md,spacy_lg,lefff,treeTagger,comparison
id,,,,,,,,,
0,Sport,Sport,Les dépenses des ménages représentent plus de ...,le dépense de ménage représenter plus de 50 po...,le dépense de ménage représenter plus de 50 po...,le dépense de ménage représenter plus de 50 po...,le dépense des ménage représenter plus de 50 p...,le dépense du ménage représenter plus de @card...,"[de|des, de|d', de|d', de|d', de|des, consacre..."
1,Jakob Böhme,Religion,"Mais ce régime de travail, combiné aux inquiét...","mais ce régime de travail , combiner à inquiét...","mais ce régime de travail , combiner à inquiét...","mais ce régime de travail , combiner à inquiét...","mais ce régime de travail , combiné aux inquié...","mais ce régime de travail , combiner au inquié...","[combiner|combiné, à|aux, générer|générées, le..."
2,Yoweri Museveni,Société,"Dans le droit fil du consensus de Washington, ...","dans le droit fil de consensus de Washington ,...","dans le droit fil de consensus de Washington ,...","dans le droit fil de consensus de Washington ,...","dans le droit fil du consensus de washington ,...","dans le droit fil du consensus de Washington ,...","[de|du, privatise|privatiser, banqu|banque, de..."
3,Les Ulis,Géographie,Le ruisseau Le Rouillon prend sa source dans l...,le ruisseau le Rouillon prendre son source dan...,le ruisseau le Rouillon prendre son source dan...,le ruisseau le Rouillon prendre son source dan...,le ruisseau le rouillon prendre son source dan...,le ruisseau le Rouillon prendre son source dan...,"[marque|marquer, dangoulêm|dangoulême, marque|..."
4,<i>Terminator 2 : Le Jugement dernier</i>,Arts,"Parallèlement, arrive du futur un autre person...","parallèlement , arrive de futur un autre perso...","parallèlement , arrive de futur un autre perso...","parallèlement , arrive de futur un autre perso...","parallèlement , arriver du futur un autre pers...","parallèlement , arriver du futur un autre pers...","[arrive|arriver, de|du, semble|sembler, de|du,..."
...,...,...,...,...,...,...,...,...,...
186,Marine indienne,Société,"La marine indienne (sanskrit : , Bharatiya Na...","le marine indien ( sanskrit : , Bharatiya na...","le marine indien ( sanskrit : , Bharatiya Na...","le marine indien ( sanskrit : , Bharatiya na...","le marine indien ( sanskrit : , bharatiya na...","le marine indien ( sanskrit : , Bharatiya Nau ...","[être|est, le|la, naval|navale, de|des, force|..."
187,Los Fabulosos Cadillacs,Arts,"Un premier album studio, intitulé Bares y fond...","un premier album studio , intituler Bares y fo...","un premier album studio , intituler bare y fon...","un premier album studio , intituler Bares y fo...","un premier album studio , intitulé bares y fon...","un premier album studio , intitulé Bares y fon...","[intituler|intitulé, intituler|intitulé, cálid..."
188,Traduction génétique,Sciences,Le processus de traduction peut être bloqué pa...,le processus de traduction pouvoir être bloque...,le processus de traduction pouvoir être bloque...,le processus de traduction pouvoir être bloque...,le processus de traduction pouvoir être bloqué...,le processus de traduction pouvoir être bloque...,"[bloquer|bloqué, bloqu|bloquer, bactérienne|ba..."
